In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler


In [6]:
# Read the dataset
data = pd.read_csv('output_trace.csv')

# Encode categorical columns using LabelEncoder
categorical_columns = ['Source UUID', 'Source type', 'Destination UUID', 'Destination type', 'Edge type']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

# Split the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the target variable (Label)
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Create a neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])


Epoch 1/30
78503/78503 [==============================] - 189s 2ms/step - loss: 0.0180 - accuracy: 0.9937 - val_loss: 0.0106 - val_accuracy: 0.9961
Epoch 2/30
78503/78503 [==============================] - 186s 2ms/step - loss: 0.0119 - accuracy: 0.9955 - val_loss: 0.0085 - val_accuracy: 0.9965
Epoch 3/30
78503/78503 [==============================] - 182s 2ms/step - loss: 0.0108 - accuracy: 0.9960 - val_loss: 0.0087 - val_accuracy: 0.9968
Epoch 4/30
78503/78503 [==============================] - 182s 2ms/step - loss: 0.0100 - accuracy: 0.9963 - val_loss: 0.0074 - val_accuracy: 0.9974
Epoch 5/30
78503/78503 [==============================] - 185s 2ms/step - loss: 0.0095 - accuracy: 0.9966 - val_loss: 0.0072 - val_accuracy: 0.9974
Epoch 6/30
78503/78503 [==============================] - 188s 2ms/step - loss: 0.0090 - accuracy: 0.9968 - val_loss: 0.0064 - val_accuracy: 0.9979
Epoch 7/30
78503/78503 [==============================] - 189s 2ms/step - loss: 0.0088 - accuracy: 0.9969 - val_

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Decode the predictions to 'Attack' or 'Normal'
y_pred = LabelEncoder().fit(['Attack', 'Normal']).inverse_transform(y_pred.ravel())

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


In [ ]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))